In [ ]:
# Preprocessing

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

There is an abundance of data. Although not all of it might be used, it's still important to inspect each dataset. Let's start with the tournament teams, tournament seeding, and regular season results.

In [ ]:
teams_m = pd.read_csv("/kaggle/input/march-madness-2025/MTeams.csv")

print("Men's tournament teams")
print(teams_m.head())

teams_w = pd.read_csv("/kaggle/input/march-madness-2025/WTeams.csv")

print("\nWomen's tournament teams")
print(teams_w.head())

seeding_m = pd.read_csv("/kaggle/input/march-madness-2025/MNCAATourneySeeds.csv")

print("\nMen's tournament seeds")
print(seeding_m.head())

seeding_w = pd.read_csv("/kaggle/input/march-madness-2025/WNCAATourneySeeds.csv")

print("\nWomen's tournament seeds")
print(seeding_w.head())

tournament_compact_results_m = pd.read_csv("/kaggle/input/march-madness-2025/MNCAATourneyCompactResults.csv")

print("\nMen's Tournament Results")
print(tournament_compact_results_m.head())

tournament_compact_results_w = pd.read_csv("/kaggle/input/march-madness-2025/WNCAATourneyCompactResults.csv")

print("\nWomen's Tournament Results")
print(tournament_compact_results_w.head())

regular_season_compact_results_m = pd.read_csv("/kaggle/input/march-madness-2025/MRegularSeasonCompactResults.csv")

print("\nMen's Regular Season Compact Results")
print(regular_season_compact_results_m.head())

regular_season_compact_results_w = pd.read_csv("/kaggle/input/march-madness-2025/WRegularSeasonCompactResults.csv")

print("\nWomen's Regular Season Compact Results")
print(regular_season_compact_results_w.head())

regular_season_detailed_results_m = pd.read_csv("/kaggle/input/march-madness-2025/MRegularSeasonDetailedResults.csv")

print("\nMen's Regular Season Compact Results")
print(regular_season_detailed_results_m.head())

regular_season_detailed_results_w = pd.read_csv("/kaggle/input/march-madness-2025/WRegularSeasonDetailedResults.csv")

print("\nMen's Regular Season Compact Results")
print(regular_season_detailed_results_w.head())

conferences_m = pd.read_csv("/kaggle/input/march-madness-2025/MTeamConferences.csv")

print("\nMen's Team Conferences")
print(conferences_m.head())

conferences_w = pd.read_csv("/kaggle/input/march-madness-2025/WTeamConferences.csv")

print("\nWomen's Team Conferences")
print(conferences_w.head())

y_actual_m = pd.read_csv("/kaggle/input/march-madness-2025/MNCAATourneyCompactResults.csv")
y_actual_w = pd.read_csv("/kaggle/input/march-madness-2025/WNCAATourneyCompactResults.csv")

In [ ]:
def get_seed(df: pd.DataFrame) -> pd.DataFrame:
    df['seed'] = df['Seed'].apply(lambda x: int(x[1:3]))
    df.drop(columns=['Seed'], inplace=True)
    return df

seeding_by_season_m = get_seed(seeding_m)
seeding_by_season_w = get_seed(seeding_w)

print(seeding_by_season_m.head())
print(seeding_by_season_w.head())

In [ ]:
def create_base_submission(df: pd.DataFrame) -> pd.DataFrame:
    # Filter rows where WTeamID < LTeamID and select relevant columns
    base1 = df.loc[df['WTeamID'] < df['LTeamID'], ['Season', 'WTeamID', 'LTeamID']]
    submission1 = base1.assign(
        ID=base1['Season'].astype(str) + "_" + base1['WTeamID'].astype(str) + "_" + base1['LTeamID'].astype(str),
        pred=1
    )
    
    # Filter rows where WTeamID > LTeamID and select relevant columns
    base2 = df.loc[df['WTeamID'] > df['LTeamID'], ['Season', 'WTeamID', 'LTeamID']]
    submission2 = base2.assign(
        ID=base2['Season'].astype(str) + "_" + base2['LTeamID'].astype(str) + "_" + base2['WTeamID'].astype(str),
        pred=0
    )
    
    # Combine and shuffle the submission formats
    submission_combined = pd.concat([submission1, submission2]).sample(frac=1.0, replace=False).reset_index(drop=True)
    
    # Drop unnecessary columns
    return submission_combined.drop(columns=['Season', 'WTeamID', 'LTeamID'])

# Create submissions for men's and women's data
submission_base_m = create_base_submission(tournament_compact_results_m)
submission_base_w = create_base_submission(tournament_compact_results_w)

# Combine submissions
submission_base_combined = pd.concat([submission_base_m, submission_base_w])

# Display the final submission
display(submission_base_combined)

The next steps are to be able to extract the win loss records of each season of each team. We need to the margin of victory, strength of schedule, and simple rating system.

In [ ]:
def margin_of_victory(df: pd.DataFrame) -> pd.DataFrame:
    # First calculate MOV for each game
    df_with_mov = df.copy()
    df_with_mov['MOV'] = df_with_mov['WScore'] - df_with_mov['LScore']

    # Create dataframe for winning teams
    winners = df_with_mov[['Season', 'WTeamID', 'MOV']].rename(columns={'WTeamID': 'TeamID'})
    winners['Win'] = 1
    winners['Loss'] = 0

    # Create dataframe for losing teams (negative MOV)
    losers = df_with_mov[['Season', 'LTeamID', 'MOV']].rename(columns={'LTeamID': 'TeamID'})
    losers['MOV'] = -losers['MOV']
    losers['Win'] = 0
    losers['Loss'] = 1

    # Combine both dataframes
    all_teams = pd.concat([winners, losers])

    # Group by Season and TeamID to get aggregate stats
    team_mov = all_teams.groupby(['Season', 'TeamID']).agg(
        avg_MOV=('MOV', 'mean'),
        total_MOV=('MOV', 'sum'),
        games_played=('MOV', 'count'),
        wins=('Win', 'sum'),
        losses=('Loss', 'sum')
    ).reset_index()

    # Calculate winning percentage
    team_mov['win_pct'] = team_mov['wins'] / team_mov['games_played']

    return team_mov

mov_m = margin_of_victory(regular_season_compact_results_m)
mov_w = margin_of_victory(regular_season_compact_results_w)

print(mov_m.head())
print(mov_w.head())    

In [ ]:
def simple_rating_system(df: pd.DataFrame, mov: pd.DataFrame) -> pd.DataFrame:
    # The mov is passed in from the margin_of_victory function, so avg_MOV is already calculated
    srs = mov.copy()

    # We initialize the SRS to the average MOV
    srs['SRS'] = srs['avg_MOV']

    # We iteratively calculate the SRS until it converges
    for i in range(100):
        # Get opponents' SRS for games where team was winner
        winner_sos = df.merge(srs[['Season', 'TeamID', 'SRS']], 
                             left_on=['Season', 'WTeamID'], 
                             right_on=['Season', 'TeamID'], 
                             how='left')
        winner_sos = winner_sos.merge(srs[['Season', 'TeamID', 'SRS']], 
                                    left_on=['Season', 'LTeamID'], 
                                    right_on=['Season', 'TeamID'], 
                                    how='left',
                                    suffixes=['_team', '_opp'])
        
        # Get opponents' SRS for games where team was loser
        loser_sos = df.merge(srs[['Season', 'TeamID', 'SRS']], 
                            left_on=['Season', 'LTeamID'], 
                            right_on=['Season', 'TeamID'], 
                            how='left')
        loser_sos = loser_sos.merge(srs[['Season', 'TeamID', 'SRS']], 
                                   left_on=['Season', 'WTeamID'], 
                                   right_on=['Season', 'TeamID'], 
                                   how='left',
                                   suffixes=['_team', '_opp'])
        
        # Combine and calculate average opponent SRS
        winner_opponents = winner_sos[['Season', 'TeamID_team', 'SRS_opp']].rename(
            columns={'TeamID_team': 'TeamID'})
        loser_opponents = loser_sos[['Season', 'TeamID_team', 'SRS_opp']].rename(
            columns={'TeamID_team': 'TeamID'})
        
        all_opponents = pd.concat([winner_opponents, loser_opponents])
        sos = all_opponents.groupby(['Season', 'TeamID'])['SRS_opp'].mean().reset_index()
        sos.columns = ['Season', 'TeamID', 'SOS']

        # Update SRS
        srs = srs.merge(sos, on=['Season', 'TeamID'], how='left')
        srs['SRS'] = srs['avg_MOV'] + srs['SOS']
        srs.drop(columns=['SOS'], inplace=True)

    return srs

srs_m = simple_rating_system(regular_season_compact_results_m, mov_m)
srs_w = simple_rating_system(regular_season_compact_results_w, mov_w)

print(srs_m.head())
print(srs_w.head())

We will now estimate possessions in order to derive Offensive Rating, Defensive Rating, and Net Rating.

The formula for deriving possessions is Possessions = 0.96 \times (FGA + TO + 0.44 \times FTA - ORB)

In [ ]:
def advanced_ratings(df: pd.DataFrame) -> pd.DataFrame:
    # Estimate possessions
    df['WPossessions'] = 0.96 * (df['WFGA'] + df['WTO'] + 0.44 * df['WFTA'] - df['WOR'])
    df['LPossessions'] = 0.96 * (df['LFGA'] + df['LTO'] + 0.44 * df['LFTA'] - df['LOR'])
    
    # Create dataframes for teams when they win
    w_teams = df[['Season', 'WTeamID', 'WScore', 'LScore', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'WPossessions', 'LPossessions']].copy()
    w_teams.columns = ['Season', 'TeamID', 'PointsScored', 'PointsAllowed', 'FGM', 'FGA', 'FGM3', 'FGA3', 'TFM', 'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF', 'OwnPossessions', 'OppPossessions']
    
    # Create dataframes for teams when they lose
    l_teams = df[['Season', 'LTeamID', 'LScore', 'WScore', 'LPossessions', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF', 'LPossessions', 'WPossessions']].copy()
    l_teams.columns = ['Season', 'TeamID', 'PointsScored', 'PointsAllowed', 'FGM', 'FGA', 'FGM3', 'FGA3', 'TFM', 'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF', 'OwnPossessions', 'OppPossessions']
    
    # Combine both datasets
    all_games = pd.concat([w_teams, l_teams])
    
    # Group by Season and TeamID to get all metrics
    team_ratings = all_games.groupby(['Season', 'TeamID']).agg(
        total_points_scored=('PointsScored', 'sum'),
        total_points_allowed=('PointsAllowed', 'sum'),
        total_field_goals_made=('FGM', 'sum'),
        total_field_goals_attempted=('FGA', 'sum'),
        total_three_point_field_goals_made=('FGM3', 'sum'),
        total_three_point_field_goals_attempted=('FGA3', 'sum'),
        total_free_throws_made=('TFM', 'sum'),
        total_free_throws_attempted=('FTA', 'sum'),
        total_offensive_rebounds=('OR', 'sum'),
        total_defensive_rebounds=('DR', 'sum'),
        total_assists=('Ast', 'sum'),
        total_turnovers=('TO', 'sum'),
        total_steals=('Stl', 'sum'),
        total_blocks=('Blk', 'sum'),
        total_PFs=('PF', 'sum'),
        total_possessions=('OwnPossessions', 'sum'),
        total_opp_possessions=('OppPossessions', 'sum'),
        games_played=('PointsScored', 'count')
    ).reset_index()
    
    # Calculate advanced metrics
    team_ratings['total_points_diff'] = team_ratings['total_points_scored'] - team_ratings['total_points_allowed']
    team_ratings['points_per_game'] = team_ratings['total_points_scored'] / team_ratings['games_played']
    team_ratings['field_goal_percentage'] = 100 * team_ratings['total_field_goals_made'] / team_ratings['total_field_goals_attempted']
    team_ratings['field_goals_made_per_game'] = team_ratings['total_field_goals_made'] / team_ratings['games_played']
    team_ratings['three_point_percentage'] = 100 * team_ratings['total_three_point_field_goals_made'] / team_ratings['total_three_point_field_goals_attempted']
    team_ratings['three_point_field_goals_made_per_game'] = team_ratings['total_three_point_field_goals_made'] / team_ratings['games_played']
    team_ratings['free_throw_percentage'] = 100 * team_ratings['total_free_throws_made'] / team_ratings['total_free_throws_attempted']
    team_ratings['offensive_rebounds_per_game'] = team_ratings['total_offensive_rebounds'] / team_ratings['games_played']
    team_ratings['defensive_rebounds_per_game'] = team_ratings['total_defensive_rebounds'] / team_ratings['games_played']
    team_ratings['rebounds_per_game'] = team_ratings['offensive_rebounds_per_game'] + team_ratings['defensive_rebounds_per_game']
    team_ratings['assists_per_game'] = team_ratings['total_assists'] / team_ratings['games_played']
    team_ratings['turnovers_per_game'] = team_ratings['total_turnovers'] / team_ratings['games_played']
    team_ratings['steals_per_game'] = team_ratings['total_steals'] / team_ratings['games_played']
    team_ratings['blocks_per_game'] = team_ratings['total_blocks'] / team_ratings['games_played']
    team_ratings['personal_fouls_per_game'] = team_ratings['total_PFs'] / team_ratings['games_played']
    team_ratings['offensive_rating'] = 100 * team_ratings['total_points_scored'] / team_ratings['total_possessions']
    team_ratings['defensive_rating'] = 100 * team_ratings['total_points_allowed'] / team_ratings['total_opp_possessions']
    team_ratings['net_rating'] = team_ratings['offensive_rating'] - team_ratings['defensive_rating']
    
    return team_ratings

ratings_m = advanced_ratings(regular_season_detailed_results_m)
ratings_w = advanced_ratings(regular_season_detailed_results_w)

print(ratings_m.head())
print(ratings_w.head())

At this point, we need to start narrowing down the variables of interest.
Here are the ones we will include:
- Season
- TeamID
- ConfAbbrev
- offensive_rating
- defensive_rating
- net_rating
- win_pct
- Seed
- Seed Difference
- rebounds_per_game
- assists_per_game
- steals_per_game
- blocks_per_game
- turnovers_per_game
- personal_fouls_per_game
- field_goal_percentage
- field_goals_made_per_game
- three_point_percentage
- three_point_field_goals_made_per_game
- total_points_scored

In [ ]:
def get_team_conference(df: pd.DataFrame, conferences: pd.DataFrame) -> pd.DataFrame:
    df = df.merge(conferences, on=['TeamID', 'Season'], how='left')
    return df

ratings_m = get_team_conference(ratings_m, conferences_m)
ratings_w = get_team_conference(ratings_w, conferences_w)

def get_seed_difference(df: pd.DataFrame, seeding: pd.DataFrame) -> pd.DataFrame:
    df = df.merge(seeding, on=['TeamID', 'Season'], how='left')
    return df

ratings_m = get_seed(ratings_m, seeding_by_season_m)
ratings_w = get_seed(ratings_w, seeding_by_season_w)

def drop_unnecessary_columns(df: pd.DataFrame) -> pd.DataFrame:
    columns = ['total_points_allowed', 'total_field_goals_made', 'total_field_goals_attempted', 
               'total_three_point_field_goals_made', 'total_three_point_field_goals_attempted', 'total_free_throws_made', 
               'total_free_throws_attempted', 'total_offensive_rebounds', 'total_defensive_rebounds', 'total_assists', 
               'total_turnovers', 'total_steals', 'total_blocks', 'total_PFs', 'total_possessions', 
               'total_opp_possessions', 'games_played']
    df.drop(columns=columns, inplace=True)
    return df

# Drop unnecessary columns
ratings_m = drop_unnecessary_columns(ratings_m)
ratings_w = drop_unnecessary_columns(ratings_w)

print(ratings_m.head())
print(ratings_w.head())

Now that we have the data, we can start to build the model using XGBoost.
The dataframes ratings_m and ratings_w should have all the relevant data we need to build the model.
But the model will need to be able to take in as input all possible matchups between two regular season teams in each season.

In [ ]:
# Import relevant libraries

# xgboost related libraries:
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import plot_importance

# sklearn related libraries:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

# Setting up the data

y_actual_m = pd.read_csv("/kaggle/input/march-madness-2025/MNCAATourneyCompactResults.csv")
y_actual_w = pd.read_csv("/kaggle/input/march-madness-2025/WNCAATourneyCompactResults.csv")

base_data_m = ratings_m.copy()
base_data_w = ratings_w.copy()

# y_actual needs to be augmented to include all possible matchups. If a matchup is not in the tourney results, then WScore will be total points scored by the winning team and LScore will be total points scored by the losing team.
# This is because the target variable is a float with range [0,1] that predicts the probability of the winning team winning the matchup.
# We'll have to use point spread to determine the target variable. Calculations should be based on betting line conventions.
# Even if two teams were matched up in the tournament, the actual variable should be a float between 0 and 1, using point spread and betting line conventions to determine the target variable.

# We will need to reshape the data so that all possible matchups are included in the data. This will be done by creating a new dataframe that includes all possible matchups between two teams in each season.

def full_matchups(df: pd.DataFrame) -> pd.DataFrame:
    df1 = df.copy()
    df2 = df.copy()
    df1.columns = ['Season', 'TeamID1', 'TeamID2']
    df2.columns = ['Season', 'TeamID2', 'TeamID1']
    df2 = df2[['Season', 'TeamID1', 'TeamID2']]
    df_full = pd.concat([df1, df2], axis=0)
    return df_full

